In [ ]:
!cp -R ../input/yolov4tool ../working

In [ ]:
from yolov4tool.utils import *
from yolov4tool.torch_utils import *
from yolov4tool.darknet2pytorch import Darknet
import argparse

In [ ]:
"""hyper parameters"""
use_cuda = True

def detect_cv2(cfgfile, weightfile, imgfile):
    import cv2
    m = Darknet(cfgfile)

    m.print_network()
    m.load_weights(weightfile)
    print('Loading weights from %s... Done!' % (weightfile))

    if use_cuda:
        m.cuda()

    num_classes = m.num_classes
    if num_classes == 20:
        namesfile = 'data/voc.names'
    elif num_classes == 80:
        namesfile = 'data/coco.names'
    else:
        namesfile = '../input/cfgdata/obj.names'
    class_names = load_class_names(namesfile)

    img = cv2.imread(imgfile)
    sized = cv2.resize(img, (m.width, m.height))
    sized = cv2.cvtColor(sized, cv2.COLOR_BGR2RGB)


    start = time.time()
    boxes = do_detect(m, sized, 0.4, 0.6, use_cuda)
    finish = time.time()
    print('%s: Predicted in %f seconds.' % (imgfile, (finish - start)))

    width = img.shape[1]
    height = img.shape[0]
    ans = ''

    b = boxes[0]
    for i in range(len(b)):
        box = b[i]
        x1 = int(box[0] * width)
        y1 = int(box[1] * height)
        x2 = int(box[2] * width)
        y2 = int(box[3] * height)
        w = x2 - x1
        h = y2 - y1
        ans = ans + str(box[4]) + ' '
        ans = ans + str(x1) + ' '
        ans = ans + str(y1) + ' '
        ans = ans + str(w) + ' '
        ans = ans + str(h) + ' '

    return ans[:-1]

In [ ]:
testfiles = [f for f in os.listdir('/kaggle/input/global-wheat-detection/test') if os.path.isfile(os.path.join('/kaggle/input/global-wheat-detection/test', f))]
box = []
name = []
for test_img in testfiles:
    cfg = '../input/cfgdata/yolov4-obj_test.cfg'
    weight = '../input/weight/yolov4-obj_6000.weights'
    img = os.path.join('/kaggle/input/global-wheat-detection/test', test_img)
    
    t = detect_cv2(cfg, weight, img)  
    name.append(test_img[:-4])
    box.append(t)


    

In [ ]:
import pandas as pd
col_list = [name, box]
sub = pd.DataFrame (col_list).transpose()
sub.columns=['image_id', 'PredictionString']
sub.to_csv('submission.csv', index=False)